In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
import json

comments = []
posts = []
for f in ['najib-razak.json', 'anwar-ibrahim.json', 'sanusi.json', 'pakatan-harapan.json',
         'rafizi-ramli.json', 'mymgag.json', 'Ayahanda-Abdul-Rani-Kulup-1753773051575006.json',
         'groups-1483603055330698.json', 'groups-392252305221999.json', 'groups-1098480360644592.json',
         'groups-2025865557592801.json']:
    with open(f) as fopen:
        data = json.load(fopen)
    
    for r in data:
        if 'groups-' not in f:
            posts.append(r['post_text'])
        for c in r['comments_full']:
            t = c['comment_text']
            if 1 < len(t.split()) < 300:
                comments.append(t)
            for c_ in c['replies']:
                t = c_['comment_text']
                if 1 < len(t.split()) < 300:
                    comments.append(t)

In [3]:
len(posts), len(comments)

(1255, 263917)

In [4]:
comments = list(set(comments))
len(comments)

199590

In [5]:
import malaya
from malaya.text.function import maxlen_slide

In [6]:
lm = malaya.language_model.kenlm(model = 'bahasa-wiki-news')

In [7]:
stemmer = malaya.stem.deep_model(model = 'noisy')

2022-12-09 15:27:34.053144: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-09 15:27:34.057547: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-12-09 15:27:34.057566: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: husein-MS-7D31
2022-12-09 15:27:34.057569: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: husein-MS-7D31
2022-12-09 15:27:34.057641: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: Not found: was unable to find libcuda.so DSO loaded into this program
2022-12-09 15:27:34.057662: I tensorflow/stream

In [8]:
corrector = malaya.spelling_correction.probability.load(language_model = lm, replace_augmentation = True,
                                                       stemmer = stemmer, maxlen = 12, minlen = 2)

In [9]:
import re

def fix_spacing_punct(string):
    string = string.replace(' , ', ', ').replace(' !', '!').replace(' . ', '. ').replace('( ', '(') \
    .replace(' )', ')')
    return re.sub(r'[ ]+', ' ', string).strip()

In [10]:
comments[0]

'Kalau UMNOBN Berkuasa , petrol tentu tak da subsidi....'

In [11]:
normalizer = malaya.normalize.normalizer(corrector, stemmer, date = False)

In [12]:
segmenter = malaya.segmentation.huggingface()
segmenter_func = lambda x: segmenter.generate([x], max_length = 256)[0]

In [13]:
text_scorer = lambda x: lm.score(x)

In [14]:
from unidecode import unidecode

unidecode('Denga kata YB rebah bing marin, semoga YB xdak apa² yg jadi laa,')

'Denga kata YB rebah bing marin, semoga YB xdak apa2 yg jadi laa,'

In [15]:
t = 'Tahniah Pm.10'
t = unidecode(t)

In [16]:
normalizer._tokenizer(t)

['Tahniah', 'Pm', '.', '10']

In [17]:
normalizer.normalize(
            t, normalize_hingga = False, normalize_cardinal = False,
            normalize_ordinal = False, normalize_pada_hari_bulan = False,
            normalize_fraction = False, normalize_money = False, normalize_date = False,
            normalize_time = False, normalize_ic = False,
            normalize_units = False,
            normalize_url = False, normalize_percent=False, normalize_telephone = False,
            segmenter = segmenter_func,
            not_a_word_threshold = 1e-9,
        )['normalize']

'Tahniah Pm . 10'

In [18]:
# true_case = malaya.true_case.huggingface()

In [19]:
# t_ = true_case.generate([t], max_length = 512)[0]
# t_

In [20]:
from tqdm import tqdm

results = []
already_in = set()
for t in tqdm(comments):

    if t.lower() in already_in:
        continue

    if not len(t):
        continue
        
    t = unidecode(t)

    try:
        normalized = normalizer.normalize(
            t, normalize_hingga = False, normalize_cardinal = False,
            normalize_ordinal = False, normalize_pada_hari_bulan = False,
            normalize_fraction = False, normalize_money = False, normalize_date = False,
            normalize_time = False, normalize_ic = False,
            normalize_units = False,
            normalize_url = False, normalize_percent=False, normalize_telephone = False,
            text_scorer = text_scorer, normalize_elongated = True,
            not_a_word_threshold = 1e-9,
        )['normalize']

        normalized = fix_spacing_punct(normalized)

        
        already_in.add(t.lower())
    except Exception as e:
        print(e)
        print(t)
        
        normalized = t
    
    results.append({'original': t, 'text': normalized})

100%|███████████████████████████████████| 199590/199590 [51:14<00:00, 64.91it/s]


In [21]:
len(results)

195959

In [22]:
results[-100:]

[{'original': 'Syukur alhamdulillah.\n\nBEST. Terbaik.\n\nTakbir!!!!!!!!!\n!',
  'text': 'Syukur alhamdulillah. Best. terbaik. takbir!!!!!!!!!!'},
 {'original': 'Diabolis Interium mancacai dap',
  'text': 'Diabolis Interium mancacai dap'},
 {'original': 'Kita sama2 sokong pemimpin yang adil, jujur, tidak rasuah dan pandai mentadbir',
  'text': 'Kita sama-sama sokong pemimpin yang adil, jujur, tidak rasuah dan pandai mentadbir'},
 {'original': 'Penjelasan mcm ni, walaun xfaham lg xtau la',
  'text': 'Penjelasan macam ini, walapun tak faham lagi tak tahu lah'},
 {'original': 'Semoga dipermudahkan..\nterbaik',
  'text': 'Semoga dipermudahkan. . terbaik'},
 {'original': 'Satu keluarga', 'text': 'Satu keluarga'},
 {'original': 'Nak terkencing hjn2 ni',
  'text': 'Nak terkencing hujan-hujan ini'},
 {'original': 'Salute rafizi', 'text': 'Sanjung rafizi'},
 {'original': 'Sound good thank you', 'text': 'Sound good thank you'},
 {'original': 'Terbaik terus bw perubahan',
  'text': 'terbaik terus

In [23]:
postss = []
for p in posts:
    postss.extend(malaya.text.function.maxlen_slide(p, maxlen = 64))
    
len(postss)

1902

In [24]:
with open('facebook-data.jsonl', 'w') as fopen:
    for r in results:
        fopen.write(f'{json.dumps(r)}\n')
    
    for p in postss:
        d = {'text': p}
        fopen.write(f'{json.dumps(d)}\n')

In [25]:
!split -l 50000 -d --additional-suffix=.splitted facebook-data.jsonl facebook-data.jsonl